In [ ]:
import argparse
from sys import platform

from models import *  # set ONNX_EXPORT in models.py
from utils.datasets import *
from utils.utils import *

In [ ]:
cfg = 'cfg/fashion.cfg'
data = 'data/fashion/fashion.data'
weights = '/home/piclick/result_backup/exp3/best.pt'

source = '/home/piclick/data/testimages/images/' 
output = 'output'
img_size = 416
conf_thres = 0.5
nms_thres = 0.3
half = False
view_img = False
device=''

In [ ]:
img_size = (320, 192) if ONNX_EXPORT else img_size
out, source, weights, half, view_img = output, source, weights, half, view_img
webcam = source == '0' or source.startswith('rtsp') or source.startswith('http') or source.endswith('.txt')
# Initialize
device = torch_utils.select_device(device='cpu' if ONNX_EXPORT else device)
if os.path.exists(out):
    shutil.rmtree(out)  # delete output folder
os.makedirs(out)  # make new output folder


In [ ]:
model = Darknet(cfg, img_size)
if weights.endswith('.pt'):  # pytorch format
    print('if')
    model.load_state_dict(torch.load(weights, map_location=device)['model'])
else:  # darknet format
    print('else')
    _ = load_darknet_weights(model, weights)

model.to(device).eval()

In [ ]:
# Set Dataloader
vid_path, vid_writer = None, None
save_txt = True
save_img = True
dataset = LoadImages(source, img_size=img_size, half=half)

# Get classes and colors
classes = load_classes(parse_data_cfg(data)['names'])
print(classes)
colors = [[random.randint(0, 255) for _ in range(3)] for _ in range(len(classes))]
print(colors)

In [ ]:
t0 = time.time()
idx = 0
for path, img, im0s, vid_cap in dataset:
    print(im0s.dtype)
    print(im0s.shape)
    
    t = time.time()
    if idx == 1000:
        break
    idx += 1

    # Get detections
    img = torch.from_numpy(img).to(device)
    if img.ndimension() == 3:
        img = img.unsqueeze(0)
    pred, _ = model(img)
    
    for i, det in enumerate(non_max_suppression(pred, conf_thres, nms_thres)):  # detections per image
        if webcam:  # batch_size >= 1
            p, s, im0 = path[i], '%g: ' % i, im0s[i]
        else:
            p, s, im0 = path, '', im0s

        save_path = str(Path(out) / Path(p).name)
        s += '%gx%g ' % img.shape[2:]  # print string
        if det is not None and len(det):
            # Rescale boxes from img_size to im0 size
            det[:, :4] = scale_coords(img.shape[2:], det[:, :4], im0.shape).round()

            # Print results
            for c in det[:, -1].unique():
                n = (det[:, -1] == c).sum()  # detections per class
                s += '%g %ss, ' % (n, classes[int(c)])  # add to string

            # Write results
            size = []
            cls_list = []
            conf_list = []
            xyxy_list = []
            
            size_threshold = 1 / 14
            boxes = len(det)
            img_height, img_width = img.shape[2:]
            print(img_width, img_height)
            print(boxes)
            
            for *xyxy, conf, _, cls in det:
                xyxy = [int(x) for x in xyxy]
                dimen = abs(xyxy[2] - xyxy[0]) * abs(xyxy[3] - xyxy[1])
                
                if cls in (4, 6, 7, 8):  # 4 bag, 6 Headwear, 7 Acc, 8 Innerwear # 1차 제거
                    print('cls')
                    boxes -= 1
                    continue
                else:  # 2차 제거 (박스 크기)
                    spatial_ratio = dimen / (img_width * img_height)

                    if spatial_ratio < size_threshold:
                        print('ratio')
                        boxes -= 1
                        continue
                
                size.append(dimen)
                xyxy_list.append(xyxy)
                conf_list.append(conf)
                cls_list.append(cls)
                
                if save_txt:  # Write to file
                    with open(save_path + '.txt', 'a') as file:
                        file.write(('%g ' * 6 + '\n') % (*xyxy, cls, conf))

                if save_img or view_img:  # Add bbox to image
                    label = '%s %.2f' % (classes[int(cls)], conf)
                    plot_one_box(xyxy, im0, label=label, color=colors[int(cls)])
                    
            if boxes == 0:
                print('no box')
            else:   
            
                max_idx = size.index(max(size))
                max_xyxy = xyxy_list[max_idx]
                conf = conf_list[max_idx]
                cls = cls_list[max_idx]

                print(boxes)
                x1,y1,x3,y3 = max_xyxy
                print(x1,y1,x3,y3)
            
            
        #print('%sDone. (%.3fs)' % (s, time.time() - t))

        # Stream results
        if view_img:
            cv2.imshow(p, im0)

        # Save results (image with detections)
        if save_img:
            if dataset.mode == 'images':
                cv2.imwrite(save_path, im0)
            else:
                if vid_path != save_path:  # new video
                    vid_path = save_path
                    if isinstance(vid_writer, cv2.VideoWriter):
                        vid_writer.release()  # release previous video writer

                    fps = vid_cap.get(cv2.CAP_PROP_FPS)
                    w = int(vid_cap.get(cv2.CAP_PROP_FRAME_WIDTH))
                    h = int(vid_cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
                    vid_writer = cv2.VideoWriter(save_path, cv2.VideoWriter_fourcc(*opt.fourcc), fps, (w, h))
                vid_writer.write(im0)

if save_txt or save_img:
    print('Results saved to %s' % os.getcwd() + os.sep + out)
    if platform == 'darwin':  # MacOS
        os.system('open ' + out + ' ' + save_path)

print('Done. (%.3fs)' % (time.time() - t0))

In [ ]:
import glob

jpg_list = glob.glob('./output/'+ '*.jpg')
png_list = glob.glob('./output/'+ '*.png')

print(len(jpg_list), len(png_list))

all_list = jpg_list + png_list

In [ ]:
de_path = '../../data/fashion_google/images/test'
r_list = []

for img in all_list:
    real_img = de_path + '/' + img.split('/')[-1]
    r_list.append(real_img)

In [ ]:
print(len(r_list))

# 2차 가장 큰 박스 값만 저장

In [ ]:
annos = []
de_path = source

for file_name in all_list:
    
    file = file_name + '.txt'
    print(file)
    try:
        f = open(file)
    except:
        continue
    out = f.readlines()
    
    anno = {}
    test = {}
    ann = []
    for bbox in range(len(out)):
        
        box = [float(x) for x in out[bbox].split(' ')[:-1]]
        ann.append(box)
        
        *xyxy, label, conf = box
        xyxy = [int(x) for x in xyxy]
        
    
        dimen = abs(xyxy[2] - xyxy[0]) * abs(xyxy[3]-xyxy[1])
        test[bbox] = dimen
    
    print(test)
        
    num = max(zip(test.keys(), test.values()), key=lambda t : t[1])[0]    
    
    anno['path'] = de_path + file_name.split('/')[-1]
    anno['xyxy'] = ann[num][:4]
    anno['label'] = ann[num][4]
    anno['conf'] = ann[num][5]

    annos.append(anno)


In [ ]:
annos[0]

# json으로 저장

In [ ]:
import json
with open('new_one_box.json', 'w') as f:
    json.dump(annos, f, ensure_ascii=False, indent = '\t')